In [1]:
import requests
import json

In [2]:
url = "http://localhost:11434/api/generate"


In [3]:
data = {
    "model": "gemma3:latest",
    "prompt": "A photo of a cat in a hat",
    "negative_prompt": "A photo of a cat in a hat, blurry, out of focus",
}

In [4]:
response = requests.post(url,json=data,stream=True)

In [5]:
if response.status_code == 200:
    with open("output.png", "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print("Image saved as output.png")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Image saved as output.png


In [3]:
!pip install -q langchain-community langchain-ollama langchain 

In [9]:


import os
import logging
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
import ollama

In [10]:
Doc_path = "./"
model_name = "gemma3:latest"
embedding_model = "nomic-embed-text"
vectorstore_path = "chroma_db"

In [15]:
def ingest_pdf(doc_path):
    loader = PyPDFLoader(doc_path)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunk = text_splitter.split_documents(data)
    return chunk

In [20]:
def create_vector(chunk):
    ollama.pull(embedding_model)
    embeddings = OllamaEmbeddings(model=embedding_model)
    vector_db = Chroma.from_documents(documents=chunk, embedding=embeddings, collection_name=vectorstore_path)
    return vector_db

In [21]:
chunk = ingest_pdf(os.path.join(Doc_path, "Test Analyst - Template 14.pdf"))
vectorstore = create_vector(chunk)  

In [23]:
vectorstore.similarity_search("what is experience in the field of data science?", k=3)

[Document(metadata={'source': './Test Analyst - Template 14.pdf', 'page_label': '1', 'creationdate': '', 'total_pages': 1, 'producer': 'Skia/PDF m102 Google Docs Renderer', 'page': 0, 'creator': 'PyPDF', 'title': 'Test Analyst - Template 14'}, page_content='Growthsi, San Francisco, CA 2013 – 2015Test Analyst\n● Implementedpredictive analytics and regression analysistechniques for quantifying durability which led toa 12% increase in the product’s overall durability.● Created effective UAT plans & scripts meeting the acceptance criteria that resulted in a 10% saving intesting time.● Developed data comparison-speciﬁc MS Excel macros and reduced the test execution time by 45%.\nResume Worded Exciting Company, San Francisco, CA 2011 – 2013Junior Test Analyst● Collaborated in the automation of the test-case design along with team members that saved an overall of55% of test design time.● Carried out an automation sanity test that preceded every deployment dropping the testing time by 40%.● Pe

In [35]:
from langchain_core.prompts import PromptTemplate

In [36]:
context = """
You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from
a vector database. By generating multiple perspectives on the user question, your
goal is to help the user overcome some of the limitations of the distance-based
similarity search. Provide these alternative questions separated by newlines.
"""

In [37]:
vectorstore.as_retriever

<bound method VectorStore.as_retriever of <langchain_community.vectorstores.chroma.Chroma object at 0x000002B85FE06C00>>

In [46]:
def create_retriever(vectorstore,llm):
    Query_prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="Given the context: {context}, answer the question: {question}",
    )

    retriever = MultiQueryRetriever.from_llm(
        vectorstore.as_retriever(), llm=llm,prompt=Query_prompt,
 # type: ignore
    )
    return retriever

In [47]:
from langchain_core.prompts import ChatPromptTemplate

In [53]:
def create_chain(retriever,llm):
    template = """
    You are an AI language model assistant. Your task is to answer the user question based on the context provided.{context} Question: {question} Answer:
        """
    prompt = ChatPromptTemplate.from_template(template)
    chain = (
        {
            "context":retriever,
            "question": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
    )
    return chain

In [54]:
from langchain_ollama  import ChatOllama

In [61]:
llm = ChatOllama(model=model_name)
retriever = create_retriever(vectorstore, llm)
chain = create_chain(retriever, llm)
query = "what is experience in the field of data science?"

In [56]:
context = "You are an AI language model assistant. Your task is to answer the user question based on the context provided."

In [59]:
response = chain.invoke(input=query, context=context)
print(response)

KeyError: "Input to PromptTemplate is missing variables {'context'}.  Expected: ['context', 'question'] Received: ['question']\nNote: if you intended {context} to be part of the string and not a variable, please escape it with double curly braces like: '{{context}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "